# Úkol č. 2 - předzpracování dat a binární klasifikace (do 2. listopadu 23:59)

  * V rámci tohoto úkolu se musíte vypořádat s příznaky, které jsou různých typů.
  * Před tím, než na nich postavíte predikční model, je třeba je nějakým způsobem převést do číselné reprezentace.
    
> **Úkoly jsou zadány tak, aby Vám daly prostor pro invenci. Vymyslet _jak přesně_ budete úkol řešit, je důležitou součástí zadání a originalita či nápaditost bude také hodnocena!**

## Zdroj dat

Budeme se zabývat predikcí přežití pasažérů Titaniku.
K dispozici máte trénovací data v souboru **data.csv** a data na vyhodnocení v souboru **evaluation.csv**.

#### Seznam příznaků:
* survived - zda přežil, 0 = Ne, 1 = Ano, **vysvětlovaná proměnná**, kterou chcete predikovat
* pclass - Třída lodního lístku, 1 = první, 2 = druhá, 3 = třetí
* name - jméno
* sex - pohlaví
* age - věk v letech
* sibsp	- počet sourozenců / manželů, manželek na palubě
* parch - počet rodičů / dětí na palubě
* ticket - číslo lodního lístku
* fare - cena lodního lístku
* cabin	- číslo kajuty
* embarked	- místo nalodění, C = Cherbourg, Q = Queenstown, S = Southampton
* home.dest - Bydliště/Cíl

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **8 bodů**:
  * V Jupyter notebooku načtěte data ze souboru **data.csv**. Vhodným způsobem si je rozdělte na podmnožiny vhodné k trénování modelu.
  * Projděte si jednotlivé příznaky a transformujte je do vhodné podoby pro použití ve vybraném klasifikačním modelu.
  * Podle potřeby si můžete vytvářet nové příznaky (na základě existujících), například tedy můžete vytvořit příznak měřící délku jména. Některé příznaky můžete také úplně zahodit.
  * Nějakým způsobem se vypořádejte s chybějícími hodnotami.
  * Následně si vyberte vhodný klasifikační model z přednášek. Najděte vhodné hyperparametry a určete jeho přesnost (accuracy) na trénovací množině. Také určete jeho přesnost na testovací množině.
  * Načtěte vyhodnocovací data ze souboru **evaluation.csv**. Napočítejte predikce pro tyto data (vysvětlovaná proměnná v nich již není). Vytvořte **results.csv** soubor, ve kterém tyto predikce uložíte do dvou sloupců: ID, predikce přežití. Tento soubor nahrajte do repozitáře.
  * Ukázka prvních řádků souboru *results.csv*:
  
```
ID,survived
1000,0
1001,1
...
```

**Další body zadání** za případné další body  (můžete si vybrat, maximum bodů za úkol je každopádně 12 bodů):
  * (až +4 body) Aplikujte všechny klasifikační modely z přednášek a určete (na základě přesnosti na validační množině), který je nejlepší. Přesnost tohoto nejlepšího modelu odhadněte pomocí křížové validace. K predikcím na vyhodnocovacích datech využijte tento model.
  * (až +4 body) Zkuste použít nějaké (alespoň dvě) netriviální metody doplňování chybějících hodnot u věku. Zaměřte na vliv těchto metod na přesnost predikce výsledného modelu. K predikcím na vyhodnocovacích datech využijte ten přístup, který Vám vyjde jako nejlepší.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte nejen Jupyter Notebook, ale i _csv_ soubor s predikcemi pro vyhodnocovací data (`results.csv`).
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni**

In [387]:
import math
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats

from sklearn import metrics, datasets
from sklearn.model_selection import ParameterGrid, train_test_split, KFold, LeaveOneOut
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import KNNImputer

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [388]:
df = pd.read_csv('data.csv')
df.drop(columns=['ID', 'name', 'ticket', 'cabin', 'home.dest', 'embarked'], inplace=True)
display(df.head())

,survived,pclass,sex,age,sibsp,parch,fare
0,1,3,male,19.0,0,0,8.0500
1,1,2,female,40.0,0,0,13.0000
2,0,3,female,18.0,0,0,6.7500
3,0,3,male,NaN,1,9,69.5500
4,0,3,female,30.0,0,0,8.6625


Find missing categoric values

In [389]:
display(df.isnull().sum(axis=0))

survived      0
pclass        0
sex           0
age         203
sibsp         0
parch         0
fare          0
dtype: int64

In [390]:
df[['sex']] = df[['sex']].astype('category').apply(lambda x: x.cat.codes)

### BONUS 1:
Use kNN for filling missing values to 'age' column.

In [391]:
n_splits = 5
random_seed = 11
imputation_err = {}
missing_cols = ['age']
col = 'age'
print('Column', col)

# Vezme si pouze kompletní data a odstraníme z nich originální odhadovanou proměnnou (SalePrice)
data_notnull = df.dropna().drop(columns='survived')

mean_imp_err = []
knn_imp_err = []
for train, val in KFold(n_splits=n_splits, random_state=random_seed).split(data_notnull.index):
    print('Fold', len(mean_imp_err) + 1, '/', n_splits)

    # Z dat odstraníme příznaky, ve kterých byly chybějící hodnoty
    Xtrain = data_notnull[data_notnull.index.isin(train)].drop(columns=missing_cols)
    # Odhaujeme vždy jeden z chybějících sloupců
    ytrain = data_notnull[data_notnull.index.isin(train)][col]
    Xval = data_notnull[data_notnull.index.isin(val)].drop(columns=missing_cols)
    yval = data_notnull[data_notnull.index.isin(val)][col]

    # Scaler opět nafitujeme podle trénovacích dat a následně transformujeme i validační
    scaler = MinMaxScaler()
    Xtrain = pd.DataFrame(scaler.fit_transform(Xtrain), index=Xtrain.index, columns=Xtrain.columns)
    Xval = pd.DataFrame(scaler.transform(Xval), index=Xval.index, columns=Xval.columns)

    # Změříme RMSLE pro doplnění průměrem
    mean_imp_err.append(np.sqrt(metrics.mean_squared_log_error(yval, np.ones(len(yval)) * ytrain.mean())))

    # Natrénujeme model pro odhad chybějícího sloupce
    model = KNeighborsRegressor(n_neighbors=5, weights='distance')
    model.fit(Xtrain, ytrain)
    # Změříme RMSLE pro doplnění pomocí kNN
    knn_imp_err.append(np.sqrt(metrics.mean_squared_log_error(yval, model.predict(Xval))))
# Zprůměrujeme chybu
imputation_err[col] = {
    'mean': np.mean(mean_imp_err),
    'knn': np.mean(knn_imp_err),
}


Column age
Fold 1 / 5
Fold 2 / 5
Fold 3 / 5
Fold 4 / 5
Fold 5 / 5


Predict age on missing data

In [392]:
data_knn = df.copy(deep=True)
X = data_knn[data_knn['age'].isna()].drop(columns=['age', 'survived'])
d = model.predict(X)
cnt = 0
for index, row in data_knn[data_knn['age'].isna()].iterrows():
    data_knn.at[index, 'age'] = d[cnt]
    cnt += 1

In [393]:
data_knn.isnull().sum(axis=0)

survived    0
pclass      0
sex         0
age         0
sibsp       0
parch       0
fare        0
dtype: int64

### Bonus 2:
Fill missing column values with distribution method, diy

Get distribution of age

In [394]:
df.age.max()
age_dist = {}
for i in range (0, 9):
    age_dist[i] = 0

Normalize age

In [395]:
total_age = 0
t = df[df.age.notna()]
sum_check = 0
for a in t.age:
    age_dist[a//10] += 1
    total_age += 1
for age in age_dist:
    age_dist[age] /= total_age
    print(age, age_dist[age])
    sum_check += age_dist[age]
print(sum_check)

0 0.08531994981179424
1 0.13801756587202008
2 0.33877038895859474
3 0.22082810539523212
4 0.11166875784190715
5 0.07026348808030113
6 0.028858218318695106
7 0.005018820577164366
8 0.0012547051442910915
1.0


Distribute age to people with NaN.

In [396]:
from numpy.random import choice
data_dist = df.copy(deep=True)
t = data_dist[data_dist.age.isnull()]
values_age = choice(list(age_dist.keys()), total_age, p = list(age_dist.values()))
it = 0

for i,row in t.iterrows():
    data_dist.at[i,'age'] = values_age[it]
    it += 1


### Data are clear
for both procedures of filling missing data

In [397]:
display(data_dist.isnull().sum(axis=0))
display(data_knn.isnull().sum(axis=0))

survived    0
pclass      0
sex         0
age         0
sibsp       0
parch       0
fare        0
dtype: int64

survived    0
pclass      0
sex         0
age         0
sibsp       0
parch       0
fare        0
dtype: int64

Apply one hot encoding

In [400]:
columns_to_category = ['sex']
data_knn = pd.get_dummies(data_knn, columns=columns_to_category) # One hot encoding the categories
data_dist = pd.get_dummies(data_dist, columns=columns_to_category) # One hot encoding the categories

In [402]:
display(data_dist)
display(data_knn)

,survived,pclass,age,sibsp,parch,fare,sex_0,sex_1
0,1,3,19.0,0,0,8.0500,0,1
1,1,2,40.0,0,0,13.0000,1,0
2,0,3,18.0,0,0,6.7500,1,0
3,0,3,2.0,1,9,69.5500,0,1
4,0,3,30.0,0,0,8.6625,1,0
...,...,...,...,...,...,...,...,...
995,0,3,4.0,0,0,7.8958,0,1
996,1,3,4.0,0,0,7.0500,0,1
997,0,3,28.0,1,1,14.4000,1,0
998,0,3,40.0,0,0,7.8958,0,1


,survived,pclass,age,sibsp,parch,fare,sex_0,sex_1
0,1,3,19.000000,0,0,8.0500,0,1
1,1,2,40.000000,0,0,13.0000,1,0
2,0,3,18.000000,0,0,6.7500,1,0
3,0,3,40.588554,1,9,69.5500,0,1
4,0,3,30.000000,0,0,8.6625,1,0
...,...,...,...,...,...,...,...,...
995,0,3,40.094098,0,0,7.8958,0,1
996,1,3,36.774385,0,0,7.0500,0,1
997,0,3,28.000000,1,1,14.4000,1,0
998,0,3,40.000000,0,0,7.8958,0,1


In [411]:
from sklearn.tree import DecisionTreeClassifier
Xdist = data_dist.iloc[:,1:]
ydist = data_dist.iloc[:,0]
Xknn = data_knn.iloc[:,1:]
yknn = data_knn.iloc[:,0]

# TRAINING

### Split data

In [412]:
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
import time

rd_seed = random_seed
XDtrain, XDtest, yDtrain, yDtest = train_test_split(Xdist, ydist, test_size=0.25, random_state=rd_seed) 
XDtrain, XDval, yDtrain, yDval = train_test_split(XDtrain, yDtrain, test_size=0.25, random_state=rd_seed) 
XKtrain, XKtest, yKtrain, yKtest = train_test_split(Xknn, yknn, test_size=0.25, random_state=rd_seed) 
XKtrain, XKval, yKtrain, yKval = train_test_split(XKtrain, yKtrain, test_size=0.25, random_state=rd_seed) 

Find best parameters

In [413]:
dtD = DecisionTreeClassifier()
dtK = DecisionTreeClassifier()
dtD.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': 'deprecated',
 'random_state': None,
 'splitter': 'best'}

In [414]:
from sklearn.model_selection import ParameterGrid
param_grid = {
    'max_depth': range(1,25), 
    'criterion': ['entropy', 'gini'],
    'splitter': ['best', 'random'],
    'min_samples': range(2,5)
}
param_comb = ParameterGrid(param_grid)

In [415]:
val_accD = []
train_accD = []
val_accK = []
train_accK = []
for params in param_comb:
    dtD = DecisionTreeClassifier(max_depth=params['max_depth'],
                                criterion=params['criterion'],splitter=params['splitter'],
                                min_samples_split=params['min_samples'])
    dtK = DecisionTreeClassifier(max_depth=params['max_depth'],
                                criterion=params['criterion'],splitter=params['splitter'],
                                min_samples_split=params['min_samples'])
    
    dtD.fit(XDtrain, yDtrain)
    dtK.fit(XKtrain, yKtrain)
    train_accD.append((metrics.accuracy_score(yDtrain, dtD.predict(XDtrain)), params))
    val_accD.append((metrics.accuracy_score(yDval, dtD.predict(XDval)), params))
    train_accK.append((metrics.accuracy_score(yKtrain, dtK.predict(XKtrain)), params))
    val_accK.append((metrics.accuracy_score(yKval, dtK.predict(XKval)), params))

In [416]:
best_valD = val_accD[0][0]
best_paramD = val_accD[0][1]
best_valK = val_accK[0][0]
best_paramK = val_accK[0][1]
best_param = best_paramK
for i in range(len(val_accD)):
    if best_valD < val_accD[i][0]:
        best_valD = val_accD[i][0]
        best_paramD = val_accD[i][1]
        print('D: ' + str(best_valD))
    if best_valK < val_accK[i][0]:
        best_valK = val_accK[i][0]
        best_paramK = val_accK[i][1]
        print('K: ' + str(best_valK))
if best_valK > best_valD:
    print("BEST IS kNN value filler\nwith accuracy: " + str(best_valK))
    best_param = best_paramK
else:
    print("BEST IS dist value filler\nwith accuracy: " + str(best_valD))
    best_param = best_paramD

D: 0.8085106382978723
K: 0.8085106382978723
K: 0.824468085106383
D: 0.8138297872340425
K: 0.8297872340425532
BEST IS kNN value filler
with accuracy: 0.8297872340425532


In [417]:
dt = DecisionTreeClassifier(max_depth=best_param['max_depth'],
    criterion=best_param['criterion'],splitter=best_param['splitter'],
    min_samples_split=best_param['min_samples'])
dt.fit(XKtrain, yKtrain)
XKtrain

,pclass,age,sibsp,parch,fare,sex_0,sex_1
366,3,21.000000,0,0,7.7333,0,1
59,3,28.000000,0,0,7.7750,1,0
152,3,38.000000,0,0,7.8958,0,1
550,3,25.000000,0,0,7.7958,0,1
701,3,0.750000,2,1,19.2583,1,0
...,...,...,...,...,...,...,...
347,3,36.591482,1,0,19.9667,0,1
34,3,16.000000,0,0,7.7333,1,0
240,1,48.000000,1,1,79.2000,1,0
71,3,22.000000,0,0,9.0000,0,1


In [418]:
print('accuracy score (train): {0:.6f}'.format(metrics.accuracy_score(yKtrain, dt.predict(XKtrain))))

accuracy score (train): 0.807829


In [419]:
print('accuracy score (test): {0:.6f}'.format(metrics.accuracy_score(yKtest, dt.predict(XKtest))))

accuracy score (test): 0.736000


# EVALUATION

Proceed with same steps of preparation as for training with kNN value filtering:
 - prepare data
 - clean data
 - divide to parts
 - evaluate

Prepare data

In [420]:
df_eval = pd.read_csv('evaluation.csv')
df_eval.drop(columns=['ID', 'name', 'ticket', 'cabin', 'home.dest', 'embarked'], inplace=True)
category_columns = ['sex']
df_eval[category_columns] = df_eval[category_columns].astype('category').apply(lambda x: x.cat.codes)
display(df_eval)

,pclass,sex,age,sibsp,parch,fare
0,2,0,24.0,2,1,27.0000
1,2,0,25.0,1,1,30.0000
2,2,1,38.0,1,0,21.0000
3,3,0,19.0,1,0,16.1000
4,2,0,60.0,1,0,26.0000
...,...,...,...,...,...,...
304,3,0,18.5,0,0,7.2833
305,3,1,32.0,0,0,56.4958
306,3,1,22.5,0,0,7.2250
307,1,0,45.0,1,1,164.8667


Clean data 

In [421]:
n_splits = 5
random_seed = 11
imputation_err = {}
missing_cols = ['age']

# Vezme si pouze kompletní data 
data_notnull = df_eval.dropna()

mean_imp_err = []
knn_imp_err = []
for train, val in KFold(n_splits=n_splits, random_state=random_seed).split(data_notnull.index):
    print('Fold', len(mean_imp_err) + 1, '/', n_splits)

    # Z dat odstraníme příznaky, ve kterých byly chybějící hodnoty
    Xtrain = data_notnull[data_notnull.index.isin(train)].drop(columns=missing_cols)
    # Odhaujeme vždy jeden z chybějících sloupců
    ytrain = data_notnull[data_notnull.index.isin(train)][col]
    Xval = data_notnull[data_notnull.index.isin(val)].drop(columns=missing_cols)
    yval = data_notnull[data_notnull.index.isin(val)][col]

    # Scaler opět nafitujeme podle trénovacích dat a následně transformujeme i validační
    scaler = MinMaxScaler()
    Xtrain = pd.DataFrame(scaler.fit_transform(Xtrain), index=Xtrain.index, columns=Xtrain.columns)
    Xval = pd.DataFrame(scaler.transform(Xval), index=Xval.index, columns=Xval.columns)

    # Změříme RMSLE pro doplnění průměrem
    mean_imp_err.append(np.sqrt(metrics.mean_squared_log_error(yval, np.ones(len(yval)) * ytrain.mean())))

    # Natrénujeme model pro odhad chybějícího sloupce
    model = KNeighborsRegressor(n_neighbors=5, weights='distance')
    model.fit(Xtrain, ytrain)
    # Změříme RMSLE pro doplnění pomocí kNN
    knn_imp_err.append(np.sqrt(metrics.mean_squared_log_error(yval, model.predict(Xval))))

# Zprůměrujeme chybu
imputation_err[col] = {
    'mean': np.mean(mean_imp_err),
    'knn': np.mean(knn_imp_err),
}

Fold 1 / 5
Fold 2 / 5
Fold 3 / 5
Fold 4 / 5
Fold 5 / 5


Predict age on missing data

In [448]:
data_knn = df_eval.copy(deep=True)
X = data_knn[data_knn['age'].isna()].drop(columns=['age'])
d = model.predict(X)
cnt = 0
for index, row in data_knn[data_knn['age'].isna()].iterrows():
    data_knn.at[index, 'age'] = d[cnt]
    cnt += 1

Bruteforce cleaning of one row

In [455]:
display(data_knn.isnull().sum(axis=0))
display(data_knn[data_knn.fare.isnull()])
median = data_knn[data_knn['pclass'] == 3].fare.median()
display(median)
for index, row in data_knn.iterrows():
    if np.isnan(row['fare']):
        data_knn.at[index, 'fare'] = median
display(data_knn.isnull().sum(axis=0))

pclass    0
age       0
sibsp     0
parch     0
fare      0
sex_0     0
sex_1     0
dtype: int64

,pclass,age,sibsp,parch,fare,sex_0,sex_1


7.8958

pclass    0
age       0
sibsp     0
parch     0
fare      0
sex_0     0
sex_1     0
dtype: int64

#### One hot encoding

In [450]:
data_knn[category_columns] = data_knn[category_columns].astype('category')
data_knn = pd.get_dummies(data_knn, columns=columns_to_category) # One hot encoding the categories

In [451]:
Xdata = data_knn.iloc[:,0:]

In [452]:
print(Xdata.isnull().sum(axis=0))

pclass    0
age       0
sibsp     0
parch     0
fare      0
sex_0     0
sex_1     0
dtype: int64


Evaluate on best learned tree

In [453]:
dt.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 5,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 4,
 'min_weight_fraction_leaf': 0.0,
 'presort': 'deprecated',
 'random_state': None,
 'splitter': 'random'}

In [454]:
# best_param = {criterion='entropy', max_depth=18, min_samples_split=4, splitter='random')
df_out = pd.DataFrame()
df_out.insert(0,'survived',dt.predict(Xdata))
df_out.insert(0,'ID',range(1000,1309))
df_out.to_csv('results.csv',index=False)

ValueError: Length of values does not match length of index